# Data Cleaning

This notebook contains the data cleaning phase of the project. The raw datasets have been downloaded from the Driven Data website. The datasets are as follows:

"test_set_values.csv": the competition testing set, containing only the id and the target variable ("status_group")

"training_set_labels.csv": the competition training set, containing the target variable

"training_set_values.csv": the competition training set, containing all the features

## Step 1
### Initial Setup

In [59]:
# importing relevant libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [60]:
# loading datasets into DataFrames
df_inspect = pd.read_csv('test_set_values.csv')
df2_inspect = pd.read_csv('training_set_labels.csv')
df3_inspect = pd.read_csv('training_set_values.csv')

The testing data set will be ignored for now, as it relates to the competition. The competition training data will be combined into a single DF for cleaning purposes, and will itself be split into testing/training/validation sets further down the line.

In [61]:
# Combining competition training sets
df4_combo = df3_inspect.join(df2_inspect, rsuffix='_r')
df5_combo = df4_combo.drop(['id_r'], axis=1)

## Step 2
### Data Exploration

The features included in the dataset are described as follows (as per the Driven Date website):

- **amount_tsh** - Total static head (amount water available to waterpoint)
- **date_recorded** - The date the row was entered
- **funder** - Who funded the well
- **gps_height** - Altitude of the well
- **installer** - Organization that installed the well
- **longitude** - GPS coordinate
- **latitude** - GPS coordinate
- **wpt_name** - Name of the waterpoint if there is one
- **num_private** -
- **basin** - Geographic water basin
- **subvillage** - Geographic location
- **region** - Geographic location
- **region_code** - Geographic location (coded)
- **district_code** - Geographic location (coded)
- **lga** - Geographic location
- **ward** - Geographic location
- **population** - Population around the well
- **public_meeting** - True/False
- **recorded_by** - Group entering this row of data
- **scheme_management** - Who operates the waterpoint
- **scheme_name** - Who operates the waterpoint
- **permit** - If the waterpoint is permitted
- **construction_year** - Year the waterpoint was constructed
- **extraction_type** - The kind of extraction the waterpoint uses
- **extraction_type_group** - The kind of extraction the waterpoint uses
- **extraction_type_class** - The kind of extraction the waterpoint uses
- **management** - How the waterpoint is managed
- **management_group** - How the waterpoint is managed
- **payment** - What the water costs
- **payment_type** - What the water costs
- **water_quality** - The quality of the water
- **quality_group** - The quality of the water
- **quantity** - The quantity of water
- **quantity_group** - The quantity of water
- **source** - The source of the water
- **source_type** - The source of the water
- **source_class** - The source of the water
- **waterpoint_type** - The kind of waterpoint
- **waterpoint_type_group** - The kind of waterpoint

The features were inspected one by one to determine if any need to be edited or removed. Below are the features for which an issue has been identified:

- **date_recorded**: deemed irrelevant for analysis - *to remove*
- **funder**: missing values, and too many single instances - *to fill and to cut down*
- **installer**: missing values, and too many single instances - *to fill and to cut down*
- **wpt_name**: deemed irrelevant for analysis - *to remove*
- **num_private**: empty feature - *to remove*
- **subvillage**: deemed irrelevant for analysis - *to remove*
- **region**: deemed irrelevant for analysis - *to remove*
- **region_code**: deemed irrelevant for analysis - *to remove*
- **district_code**: deemed irrelevant for analysis - *to remove*
- **lga**: deemed irrelevant for analysis - *to remove*
- **ward**: deemed irrelevant for analysis - *to remove*
- **public_meeting**: deemed irrelevant for analysis - *to remove*
- **recorded_by**: all values are the same, none missing - *to remove*
- **scheme_management**: missing values, duplicate of 'management' - *to remove*
- **scheme_name**: missing values, deemed irrelevant for analysis - *to remove*
- **permit**: missing values - *to fill*
- **construction_year**: mostly zero; not useful for analyis - *to convert to age in months and fill missing with median*
- **extraction_type_group** - subsumed by 'extraction_type' - *to remove*
- **extraction_type_class** - subsumed by 'extraction_type' - *to remove*
- **payment_type**: duplicate of 'payment' - *to remove*
- **water_quality**: subsumed by 'quality_group' - *to remove*
- **quantity_group**: duplicate of 'quantity' - *to remove*
- **source_type**: subsumed by 'source' - *to remove*
- **source_class**: subsumed by 'source' - *to remove*
- **waterpoint_type_group**: subsumed by 'waterpoint_type' - *to remove*

A specific point to mention is that the 'funder' and 'installer' variables have many singular instances, ie there are many organisations that have funded/installed a single well. This means that there is no useful insight to be gained from these features as they are, since a testing data point will probably not fit into any existing category here. To solve this, these two features will be limited to the top 6 installers and funders, and all remaining values will be grouped into a single value named 'small_org'.

## Step 3
### Processing

A function is created in order to clean the entire dataset. The function is documented below, and can also be used to clean the competition test set, if required.

In [62]:
# creating function to clean and rearrange dataframe

def preprocess_clean(dataframe, cols_to_drop, cols_to_fill, col_order, target_name):
    
    """Function to clean and rearrange dataframe for use in the model.
    
    The function will take the dataframe as an argument and perform the following operations:
    - Fill 'construction year' nil values with median non-zero value
    - Convert 'construction year' to age in years
    - Remove the rows identified as irrelevant
    - Fill categorical variable NAs with 'unknown'
    - Limit funders and installers to top five, with remaining values grouped into single value
    - Rearranges column order for readibility
    
    Args:
        'dataframe': the dataframe to clean
        'cols_to_drop': list of columns to drop from the dataframe
        'cols_to_fill': list of categorical columns to fill
        'col_order': list of columns in the desired order for the output
        'target_name': name (as string) of target variable
    
    Returns:
        A cleaned dataframe with the desired columns in the order specified
    """
    
    # creating copy of dataframe
    df = dataframe.copy()
    
    # converting 'construction_year' to age in years and filling zeros with median
    if 'construction_year' in df.columns:
        df.replace(0, np.nan, inplace=True)
        median_x = df['construction_year'].median(skipna=True)
        df['construction_year'].fillna(median_x, inplace=True)
        df['age_years'] = round(2019 - df['construction_year'],0)

    else:
        pass

    # dropping columns identified above as irrelvant
    df2 = df.drop(cols_to_drop, axis=1)
    
    # filling missing values with 'unknown' categorical
    df2[cols_to_fill] = df2[cols_to_fill].fillna(value='unknown')
    
    # limiting installers to top 6 only, with remaining values grouped together
    if 'installer' in df2.columns:
        installer_series = df2['installer'].value_counts()
        installer_series_2 = installer_series.sort_values(ascending=False)
        installer_top_6 = installer_series_2.index[0:6]
        installer_top_6_list = list(installer_top_6)
        df2['installer'] = df2['installer'].apply(lambda x: x if x in installer_top_6_list else 'small_inst')
    
    else:
        pass

    # limiting funders to top 6 only, with remaining values grouped together
    if 'funder' in df2.columns:
        funder_series = df2['funder'].value_counts()
        funder_series_2 = funder_series.sort_values(ascending=False)
        funder_top_6 = funder_series_2.index[0:6]
        funder_top_6_list = list(funder_top_6)
        df2['funder'] = df2['funder'].apply(lambda x: x if x in funder_top_6_list else 'small_fund')
    
    else:
        pass    
    
    # rearranging column order, and excluding target variable if not in dataframe
    if target_name in df2.columns:
        df3 = df2[col_order]
    else:
        list2 = order_cols[:order_cols.index(target_name)]+order_cols[order_cols.index(target_name)+1:]
        df3 = df2[list2]
    
    df4 = df3.fillna(0)
    
    return df4
    

In [63]:
# creating list of columns to drop, in order to pass into the cleaning function
drop_cols = ['date_recorded',
                'wpt_name',
                'num_private',
                'subvillage',
                'region',
                'region_code',
                'district_code',
                'lga',
                'ward',
                'public_meeting',
                'recorded_by',
                'scheme_management',
                'scheme_name',
                'construction_year',
                'extraction_type_group',
                'extraction_type_class',
                'payment_type',
                'water_quality',
                'quantity_group',
                'source_type',
                'source_class',
                'waterpoint_type_group']

# creating list of columns to fill with 'unknown'
fill_cols = ['funder',
            'installer',
            'permit']

# creating list of columns order
order_cols = ['id',
            'amount_tsh',
            'funder',
            'installer',
            'gps_height',
            'longitude',
            'latitude',
            'basin',
            'population',
            'permit',
            'extraction_type',
            'management',
            'management_group',
            'payment',
            'quality_group',
            'quantity',
            'source',
            'waterpoint_type',
            'age_years',
            'status_group',]

In [67]:
# cleaning the datatset using the cleaning function
df_clean = preprocess_clean(df5_combo, drop_cols, fill_cols, order_cols, 'status_group')
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                59400 non-null  float64
 1   amount_tsh        59400 non-null  float64
 2   funder            59400 non-null  object 
 3   installer         59400 non-null  object 
 4   gps_height        59400 non-null  float64
 5   longitude         59400 non-null  float64
 6   latitude          59400 non-null  float64
 7   basin             59400 non-null  object 
 8   population        59400 non-null  float64
 9   permit            59400 non-null  object 
 10  extraction_type   59400 non-null  object 
 11  management        59400 non-null  object 
 12  management_group  59400 non-null  object 
 13  payment           59400 non-null  object 
 14  quality_group     59400 non-null  object 
 15  quantity          59400 non-null  object 
 16  source            59400 non-null  object

In [70]:
df_clean_2 = preprocess_clean(df_inspect, drop_cols, fill_cols, order_cols, 'status_group')
df_clean_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                14850 non-null  int64  
 1   amount_tsh        14850 non-null  float64
 2   funder            14850 non-null  object 
 3   installer         14850 non-null  object 
 4   gps_height        14850 non-null  float64
 5   longitude         14850 non-null  float64
 6   latitude          14850 non-null  float64
 7   basin             14850 non-null  object 
 8   population        14850 non-null  float64
 9   permit            14850 non-null  object 
 10  extraction_type   14850 non-null  object 
 11  management        14850 non-null  object 
 12  management_group  14850 non-null  object 
 13  payment           14850 non-null  object 
 14  quality_group     14850 non-null  object 
 15  quantity          14850 non-null  object 
 16  source            14850 non-null  object

In [71]:
df_clean.to_csv('df_clean.csv')